In [ ]:
class UnreachableCodeVisitor(ast.NodeVisitor):
    def __init__(self):
        self.unreachable_nodes = []  # List of unreachable nodes
        self.scope_stack = []  # Stack to track current scope
        self.reachable = True  # Indicates if the current code is reachable

    def visit_FunctionDef(self, node): # Function to visit a function definition 
        # Reset reachability when entering a function
        self.scope_stack.append(node.name)
        self.reachable = True
        self.generic_visit(node)
        self.scope_stack.pop()

    def visit_Return(self, node): # Function to visit a return statement
        # After a return, subsequent code is unreachable within the same scope
        if self.reachable:
            self.reachable = False
        self.generic_visit(node)

    def visit_Break(self, node): # Function to visit a break statement
        # After a break, subsequent code in the same loop is unreachable
        if self.reachable:
            self.reachable = False
        self.generic_visit(node) # Continue checking reachable code

    def visit_Continue(self, node): # Function to visit a continue statement
        # After a continue, subsequent code in the same iteration is unreachable
        if self.reachable:
            self.reachable = False 
        self.generic_visit(node) # Continue checking reachable code

    def visit_If(self, node): # Function to visit an if statement
        # For conditional blocks, visit each branch independently
        self.generic_visit(node)  # Continue checking reachable code

    def visit_Assign(self, node): # Function to visit an assignment statement
        # Flag assignments as unreachable if they are in unreachable code
        if not self.reachable:
            self._detect_unreachable_code(node)
        self.generic_visit(node)

    def visit_Expr(self, node): # Function to visit an expression statement
        # Flag expressions (e.g., print statements) as unreachable if necessary
        if not self.reachable:
            self._detect_unreachable_code(node)
        self.generic_visit(node)

    def _detect_unreachable_code(self, node): # Function to detect unreachable code
        # Record the unreachable node with its scope and line number
        if self.scope_stack and not self.reachable:
            self.unreachable_nodes.append((self.scope_stack[-1], node.lineno))
# Function to test multiple cases
def test_code(code):
    tree = ast.parse(code)
    unreachable_visitor = UnreachableCodeVisitor()
    unreachable_visitor.visit(tree)

    unreachable_checker = UnreachableCodeChecker()
    unreachable_checker.check_unreachable(unreachable_visitor.unreachable_nodes)

    print("Unreachable code violations:", unreachable_checker.violations)

# Test cases
test_cases = [
    ("Test Case 1", """
def exampleFunction():
    x = 5
    y = 10
    return x
"""),
    ("Test Case 2", """
def exampleFunction():
    x = 5
    return x
    y = 10  # Unreachable
"""),
    ("Test Case 3", """
def exampleFunction():
    for i in range(5):
        if i == 2:
            break
        print(i)
    print('Unreachable')  # Unreachable
"""),
    ("Test Case 4", """
def exampleFunction():
    x = 5
    if x > 0:
        return x
    print('Unreachable')  # Unreachable
"""),
    ("Test Case 5", """
def exampleFunction():
    for i in range(5):
        if i % 2 == 0:
            continue
            print('Unreachable')  # Unreachable
        print(i)
"""),
    ("Test Case 6", """
def exampleFunction(): 
    x = 5
    for i in range(x):
        print(i)
    return x
""")
]

for name, code in test_cases:
    print(name)
    test_code(code)
    print()


Test Case 1
Unreachable code violations: set()

Test Case 2
Unreachable code violations: {"Violation: Unreachable code detected in 'exampleFunction' at line 5."}

Test Case 3
Unreachable code violations: {"Violation: Unreachable code detected in 'exampleFunction' at line 6.", "Violation: Unreachable code detected in 'exampleFunction' at line 7."}

Test Case 4
Unreachable code violations: {"Violation: Unreachable code detected in 'exampleFunction' at line 6."}

Test Case 5
Unreachable code violations: {"Violation: Unreachable code detected in 'exampleFunction' at line 6.", "Violation: Unreachable code detected in 'exampleFunction' at line 7."}

Test Case 6
Unreachable code violations: set()

